In [7]:
import os
import numpy as np
import pandas as pd
from random import *
import json
import joblib
from IPython.display import display


from app import jsonify, Response
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

def defaultOptimize():
    df_00 = pd.read_csv("./boston_house.csv")


    ''' 변수 1 (MEDV)'''
    MEDV = df_00['MEDV']

    ''' X (그 외)'''
    X = df_00.drop(['MEDV','CRIM'], axis = 1)
    
    X_mean = X.mean().round(2) #그 외 변수의 mean (평균)
    X_std = X.std().round(1)*0.1 #그 외 변수의 std (표준편차)

    X_info = pd.DataFrame({'mean':X_mean.values,'std':X_std.values})
    randNumbers = []

    # 난수를 생성함과 동시에 series로 형변환해서 randNumbers에 삽입
    # randNumbers: series 난수들을 갖고 있는 배열
    #  
    for i,v in X_info.iterrows():
        mean = v[0]
        std = v[1]
        randomized = np.random.normal(mean,std*0.01,1000)
        randomized = pd.Series(randomized)
        randNumbers.append(randomized)

    X_final = pd.DataFrame(randNumbers)
    X_final = X_final.transpose() # 컬럼과 인덱스 순서 변경
    X_final.columns = X.columns # 컬럼명 주기
    X_final['RAD'] = pd.Series(np.random.randint(df_00['RAD'].min(),df_00['RAD'].max()*0.8,1000))
    X_final['CHAS'] = pd.Series(np.random.randint(0,1,1000))
    display(X_final)
  

    ''' CRIM '''
    CRIM = df_00['CRIM']

    CRIM_min = CRIM.min()
    CRIM_max = CRIM.max()
    # print('y_max',y_max)
    CRIM_rand = np.random.uniform(CRIM_min,CRIM_max,1000)
    CRIM_final = pd.Series(CRIM_rand)
    X_final['CRIM'] = CRIM_final

    ## NORMALIZATION ##
    file_name = 'scaler_xgboost.pkl'
    scalerX= joblib.load(file_name) 
    X_final_n = scalerX.transform(X_final)


    '''시작'''

    xgb_model = xgb.XGBRegressor(n_estimators = 500, 
                            learning_rate = 0.08, 
                            gamma = 0.3, 
                            eta = 0.04,
                            subsample = 0.75,
                            colsample_bytree = 0.5, 
                            max_depth = 7)

    xgb_model.load_model("model.json")
    xgb_model_predict_x = xgb_model.predict(X_final_n)

    X_final['MEDV'] = xgb_model_predict_x
    totalFinal = X_final.sort_values(by='CRIM')
    print(totalFinal)
    

    return jsonify({"MEDV_predict":totalFinal['MEDV'].tolist(),"CRIM_simulated":totalFinal['CRIM'].tolist()})

ModuleNotFoundError: No module named 'numpy'

In [10]:
! ls

Untitled.ipynb         __pycache__            defaultOptimization.py
